In [3]:
import sys
import os

# Get the project root (adjust path as needed)
project_root = os.path.abspath("..")   # one folder up from current notebook
sys.path.append(project_root)



import sys
import os

#### Add parent folder to path so we can import resume_skill_extractor
sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__), "..")))

from resume_skill_extractor.skill_extractor import extract_skills


In [ ]:
from resume_skill_extractor.skill_extractor import extract_skills_cached 
from langchain_community.embeddings import OllamaEmbeddings

embeddings=(
    OllamaEmbeddings(model="gemma:2b")        ##by default it use llama2
)


C:\Users\sneha\AppData\Local\Temp\ipykernel_16372\2025362762.py:5: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  OllamaEmbeddings(model="gemma:2b")        ##by default it use llama2


In [ ]:
from resume_skill_extractor.skill_extractor import extract_skills_cached
from langchain_community.embeddings import HuggingFaceEmbeddings
import numpy as np

# Initialize embeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

def embed_skills(resume_skills, jd_skills):
    """
    Embed each skill separately (one vector per skill).
    """
    if not isinstance(resume_skills, list):
        raise ValueError("resume_skills must be a list of strings")
    if not isinstance(jd_skills, list):
        raise ValueError("jd_skills must be a list of strings")

    resume_vectors = embeddings.embed_documents(resume_skills)
    jd_vectors = embeddings.embed_documents(jd_skills)

    return resume_vectors, jd_vectors


def cosine_sim(vec1, vec2):
    dot = np.dot(vec1, vec2)
    mag_vec1 = np.linalg.norm(vec1)
    mag_vec2 = np.linalg.norm(vec2)
    return dot / (mag_vec1 * mag_vec2)


if __name__ == "__main__":
    # Example input text (later you’ll load real resume/JD text)           ### I have to add extracted skills from module 1. first do this 
    resume_text = "Resume text with Python, SQL, Machine Learning..."
    jd_text = "JD requires Python, SQL, Deep Learning..."

    # 🔹 Call Module 1 functions
    resume_skills,jd_skills = extract_skills_cached(resume_text,jd_text,model_choice="llama3-8b-8192")
   

    # 🔹 Embed and compare
    resume_vectors, jd_vectors = embed_skills(resume_skills, jd_skills)

    for i, r_vec in enumerate(resume_vectors):
        for j, j_vec in enumerate(jd_vectors):
            sim = cosine_sim(r_vec, j_vec)
            print(f"Similarity between Resume skill '{resume_skills[i]}' and JD skill '{jd_skills[j]}': {sim:.4f}")



Similarity between Resume skill 'Python' and JD skill 'Python': 1.0000
Similarity between Resume skill 'Python' and JD skill 'SQL': 0.3000
Similarity between Resume skill 'Python' and JD skill 'Deep Learning': 0.3113
Similarity between Resume skill 'SQL' and JD skill 'Python': 0.3000
Similarity between Resume skill 'SQL' and JD skill 'SQL': 1.0000
Similarity between Resume skill 'SQL' and JD skill 'Deep Learning': 0.2231
Similarity between Resume skill 'Machine Learning' and JD skill 'Python': 0.3613
Similarity between Resume skill 'Machine Learning' and JD skill 'SQL': 0.3169
Similarity between Resume skill 'Machine Learning' and JD skill 'Deep Learning': 0.6887


In [1]:
def find_matches(resume_skills, jd_skills, resume_vectors, jd_vectors, threshold=0.7):
    matched_skills = []
    missing_skills = []

    for j, jd_skill in enumerate(jd_skills):
        best_sim = 0
        best_resume_skill = None

        for i, resume_skill in enumerate(resume_skills):
            sim = cosine_sim(resume_vectors[i], jd_vectors[j])
            if sim > best_sim:
                best_sim = sim
                best_resume_skill = resume_skill

        if best_sim >= threshold:
            matched_skills.append((jd_skill, best_resume_skill, best_sim))
        else:
            missing_skills.append(jd_skill)

    score = len(matched_skills) / len(jd_skills)
    return matched_skills, missing_skills, score

In [2]:
if __name__ == "__main__":
    resume = ["Python", "SQL", "Machine Learning"]
    jd = ["Python", "SQL", "Deep Learning"]

    resume_vecs, jd_vecs = embed_skills(resume, jd)
    matches, missing, score = find_matches(resume, jd, resume_vecs, jd_vecs)

    print("✅ Matches:", matches)
    print("❌ Missing:", missing)
    print("📊 Score:", score)


NameError: name 'embed_skills' is not defined

In [17]:
import sys
import os
project_root = os.path.abspath("..")   # one folder up from current notebook
sys.path.append(project_root)
from resume_skill_extractor.skill_extractor import extract_skills_cached 
from resume_skill_extractor.app import resume_text,jd_text
from langchain_community.embeddings import OllamaEmbeddings
import numpy as np




##Initialize Embedding model
embeddings=(
    OllamaEmbeddings(model="gemma:2b")        ##by default it use llama2
)


# Initialize embeddings
def embed_skills(resume_skills, jd_skills):
    """
    Embed each skill separately (one vector per skill).
    """
    if not isinstance(resume_skills, list):
        raise ValueError("resume_skills must be a list of strings")
    if not isinstance(jd_skills, list):
        raise ValueError("jd_skills must be a list of strings")

    resume_vectors = embeddings.embed_documents(resume_skills)
    jd_vectors = embeddings.embed_documents(jd_skills)

    return resume_vectors, jd_vectors


def cosine_sim(vec1, vec2):
    dot = np.dot(vec1, vec2)
    mag_vec1 = np.linalg.norm(vec1)
    mag_vec2 = np.linalg.norm(vec2)
    return dot / (mag_vec1 * mag_vec2)


def find_matches(resume_skills, jd_skills, resume_vectors, jd_vectors, threshold=0.7):
    if not jd_skills:
        return [], [], 0.0
    

    matched_skills = []
    missing_skills = []

    for j, jd_skill in enumerate(jd_skills):
        best_sim = 0
        best_resume_skill = None

        for i, resume_skill in enumerate(resume_skills):
            sim = cosine_sim(resume_vectors[i], jd_vectors[j])
            if sim > best_sim:
                best_sim = sim
                best_resume_skill = resume_skill

        if best_sim >= threshold:
            matched_skills.append((jd_skill, best_resume_skill, round(best_sim, 3)))
        else:
            missing_skills.append(jd_skill)

    score = round((len(matched_skills) / len(jd_skills)) * 100, 2)

    return matched_skills, missing_skills, score

if __name__ == "__main__":

    # Correct unpacking
    resume_skills, jd_skills = extract_skills_cached(resume_text, jd_text, model_choice="mistral:latest")

    # Now this will work
    resume_vectors, jd_vectors = embed_skills(resume_skills, jd_skills)
    matches, missing, score = find_matches( resume_vecs,resume_vectors,jd_vectors, jd_vecs)

    print("✅ Matches:", matches)

    print("❌ Missing:", missing)
    print("📊 Score:", score)


✅ Matches: []
❌ Missing: []
📊 Score: 0.0
